In [ ]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from notears import NOTEARS
from notears.data import generate_continuous_dag, generate_random_data
from notears.loss import LinearSEMLoss

import matplotlib.pyplot as plt
import seaborn as sns

# Set seed for reproducability
np.random.seed(122)

# Data Inspection

In [ ]:
"""
Functions for plotting
"""

def plot_data(data):
    _, nodes = data.shape
    df_data = pd.DataFrame(
        data=data, columns=[f'X{i+1}' for i in range(nodes)]
    )
    fig = sns.pairplot(df_data, height=2, plot_kws={'s':1.5})
    fig.figure.suptitle('Pairplot of Data')
    fig.tight_layout()

def plot_dag(dag, ax=None, cmap='Grays'):
    ax = plt.gca() if ax==None else ax
    nodes, _ = dag.shape
    im = ax.imshow(dag, cmap=cmap)
    plt.colorbar(im, ax=ax)
    ax.set_yticks([i for i in range(nodes)], [f'X{i+1}' for i in range(nodes)])
    ax.set_xticks([i for i in range(nodes)], [f'X{i+1}' for i in range(nodes)])
    ax.set_title('Visual Representation of Adjacency Matrix')

In [ ]:
nodes, sparsity = 4, 0.5
dag = generate_continuous_dag(nodes, sparsity)
plot_dag(dag)

In [ ]:
data = generate_random_data(dag, 1000, noise_scale=5)
plot_data(data)

# Association $\neq$ Causation

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
coef_matrix = np.zeros_like(dag)
for var in range(dag.shape[0]):
    other_vars = [other for other in range(dag.shape[0]) if other != var]
    lr.fit(X=data[:, other_vars], y=data[:, var])
    coef_matrix[other_vars, var] = lr.coef_

plot_dag(coef_matrix)

In [ ]:
nt = NOTEARS(0, 1e-8, 0.25, LinearSEMLoss())
nt.fit(data)
dag_found = nt.get_discovered_graph()

plot_dag(dag_found)

# Markov Equivalence Class
Let's demonstrate when NOTEARS fails to find a unique solution:

In [ ]:
from joblib import Parallel, delayed


def run(data:np.ndarray, nt:NOTEARS):
    result = nt.fit(data).get_discovered_graph()
    return result

def repeated_run(nt:NOTEARS, data:np.ndarray, n_runs:int) -> np.ndarray:
    results = Parallel(n_jobs=-1)(
        delayed(run)(data, nt) for _ in tqdm(range(n_runs), desc='Running Test', leave=False)
    )
    return results

In [ ]:
dims = 4
samples = 100

custom_data = pd.DataFrame(
    columns=[f'X{i+1}' for i in range(dims)],
    data = np.random.normal(scale=2, size=(samples, dims))
)

# CREATE CUSTOM DEPENDENCIES
custom_data['X1'] = custom_data['X2'] + custom_data['X4'] + #np.random.normal(scale=3, size=samples)
custom_data['X3'] = custom_data['X1'] + #np.random.normal(scale=3, size=samples)

plot_data(custom_data.values)

In [ ]:
nt = NOTEARS(0, 1e-8, 0.25, LinearSEMLoss())
results = repeated_run(nt, custom_data.values, n_runs=16)

fig, axs = plt.subplots(4, 4)
for i, result in enumerate(results):
    plot_dag(result, ax=axs[i//4, i%4])
    axs[i//4, i%4].set_title('')

fig.suptitle('Demonstration Markov Equivalence Classes')
fig.tight_layout()
plt.show()